In [1]:
import numpy as np
import pandas as pd
import csv
import pickle
import math
import cmath
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from random import randint
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import scipy as sp
from os.path import dirname, join as pjoin
import scipy.io as sio
from matplotlib import mlab
from tqdm import tqdm
import time
import pathlib
import csv
import os
import warnings
warnings.filterwarnings("ignore")
import glob

/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
print(os.getcwd())
directory='/home/zandi.r/Data/8 Class/'

/home/zandi.r/Data


In [5]:
def get_CSIs(directory, train_speeds, test_speed):
    CSIs = []
    Labels = []
    CSIs_test = []
    Labels_test = []

    for labels in os.listdir(directory):  # Main Directory where each class label is present as a folder name.
        label_path = os.path.join(directory, labels)
        if os.path.isdir(label_path):
            if labels == 'Arc':  # Folder contains ARCXZ CSIs get the '0' class label.
                label = 0
            elif labels == 'Elbow':
                label = 1
            elif labels == 'Rectangle':
                label = 2
            elif labels == 'Silence':
                label = 3  
            elif labels == 'SLFW':
                label = 4
            elif labels == 'SLRL':
                label = 5
            elif labels == 'SLUD':
                label = 6
            elif labels == 'Triangle':
                label = 7

            # Get a list of speeds in the label directory
            speed_folders = os.listdir(label_path)
            for speed_folder in speed_folders:
                if speed_folder in train_speeds:
                    data_type = 'train'
                elif speed_folder == test_speed:
                    data_type = 'test'
                else:
                    continue

                speed_path = os.path.join(label_path, speed_folder)
                for csi_file in os.listdir(speed_path):  # Extracting the file name of the csi from Speed folder
                    if csi_file == '.DS_Store':
                        continue  # Skip processing this file
                    def read_file(filename):
                        with open(filename, 'rb') as FID:
                            mp = pickle.Unpickler(FID)
                            data = mp.load()
                        return data

                    csi = read_file(os.path.join(speed_path, csi_file))  # Reading the csi
                    ar1 = np.asmatrix(np.abs(csi[0]['complex_csi']))
                    arr1 = np.delete(ar1, [0, 1, 2, 3, 4, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 253, 254, 255], 1)
                    arr1 = np.pad(arr1, ((0, 0), (64, 64)), mode='constant')

                    ar2 = np.asmatrix(np.abs(csi[1]['complex_csi']))
                    arr2 = np.delete(ar2, [0, 1, 2, 3, 4, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 253, 254, 255], 1)
                    arr2 = np.pad(arr2, ((0, 0), (64, 64)), mode='constant')

                    arr = np.concatenate((arr1, arr2), axis=1)

                    # Append to the dataset based on data_type
                    if data_type == 'train':
                        # Append to training data
                        CSIs.append(arr)
                        Labels.append(label)
                    elif data_type == 'test':
                        # Append to testing data
                        CSIs_test.append(arr)
                        Labels_test.append(label)

    return shuffle(CSIs, Labels, random_state=817328462), shuffle(CSIs_test, Labels_test, random_state=817328462)

# def get_classlabel(class_code):
#     labels = {0: 'ARC', 1: 'DIAG_XY', 2: 'Elbow', 3: 'Rectangle', 4: 'Silence', 5: 'SLFW', 6: 'SLRL', 7: 'SLUD'}
#     return labels[class_code]

In [6]:
# Define train and test speeds
train_speeds = ['V1','V3']  # Two speeds for training
test_speed = 'V2'  # One speed for testing

# Call the function to get training and testing data
training_data, testing_data = get_CSIs(directory, train_speeds, test_speed)

# Separate training and testing data
train_CSIs, train_labels = training_data
test_CSIs, test_labels = testing_data


In [ ]:
train_CSIs=np.array(train_CSIs)
train_labels=np.array(train_labels)
print("Shape of Images:",train_CSIs.shape)
print("Shape of Labels:",train_labels.shape)